# Vision3D Tutorial: 3D Reconstruction from Images

This notebook demonstrates how to use Vision3D for 3D reconstruction from a collection of images.

## Prerequisites

Make sure you have installed Vision3D:
```bash
pip install -e .
```

## 1. Setup and Imports

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import logging

from vision3d import Vision3DPipeline
from vision3d.models import LoFTRMatcher, SuperGlueMatcher
from vision3d.utils.visualization import visualize_matches, visualize_reconstruction

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check GPU availability
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## 2. Load Sample Images

For this tutorial, we'll use a sample dataset. Replace with your own images.

In [ ]:
# Define image directory
image_dir = Path('./sample_images')  # Replace with your image directory
image_paths = sorted(list(image_dir.glob('*.jpg')))

print(f"Found {len(image_paths)} images")

# Display sample images
fig, axes = plt.subplots(1, min(4, len(image_paths)), figsize=(20, 5))
for i, img_path in enumerate(image_paths[:4]):
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[i].imshow(img_rgb)
    axes[i].set_title(img_path.name)
    axes[i].axis('off')
plt.tight_layout()
plt.show()

## 3. Basic 3D Reconstruction

Let's perform a basic reconstruction using the default settings.

In [ ]:
# Initialize pipeline
pipeline = Vision3DPipeline(matcher_type='hybrid', device=device)

# Run reconstruction
reconstruction = pipeline.reconstruct(
    [str(p) for p in image_paths],
    output_dir='./output/basic',
    verbose=True
)

In [ ]:
# Display reconstruction statistics
stats = reconstruction['statistics']
print("\nReconstruction Statistics:")
print(f"  Registered images: {stats['num_images']} / {len(image_paths)}")
print(f"  3D points: {stats['num_points3D']}")
print(f"  Mean reprojection error: {stats['mean_reprojection_error']:.3f} px")
print(f"  Mean track length: {stats['mean_track_length']:.1f}")

if stats['point_cloud_bounds']:
    bounds = stats['point_cloud_bounds']
    print(f"\nPoint cloud bounds:")
    print(f"  Min: {bounds['min']}")
    print(f"  Max: {bounds['max']}")
    print(f"  Center: {bounds['center']}")

## 4. Custom Configuration

Let's try reconstruction with custom settings optimized for quality.

In [ ]:
# Custom configuration for high-quality reconstruction
custom_config = {
    'image_resize': 1440,  # Higher resolution
    'pair_selection': {
        'min_pairs': 30,
        'similarity_threshold': 0.5
    },
    'matching': {
        'confidence_threshold': 0.3,
        'use_tta': True,
        'tta_variants': ['orig', 'flip_lr', 'rot_90']
    },
    'reconstruction': {
        'min_model_size': 3,
        'ba_refine_focal': True,
        'ba_refine_principal': True,
        'ba_refine_distortion': True
    }
}

# Initialize pipeline with custom config
pipeline_custom = Vision3DPipeline(
    matcher_type='hybrid',
    device=device,
    config=custom_config
)

# Run reconstruction
reconstruction_custom = pipeline_custom.reconstruct(
    [str(p) for p in image_paths],
    output_dir='./output/custom',
    verbose=True
)

## 5. Feature Matching Visualization

Let's visualize the feature matches between a pair of images.

In [ ]:
# Select two images to match
img1_path = image_paths[0]
img2_path = image_paths[1]

# Initialize matcher
matcher = LoFTRMatcher(device=device)

# Match features
kpts1, kpts2, conf = matcher.match_pair(str(img1_path), str(img2_path))

print(f"Found {len(kpts1)} matches")
print(f"Mean confidence: {np.mean(conf):.3f}")

In [ ]:
# Visualize matches
img1 = cv2.imread(str(img1_path))
img2 = cv2.imread(str(img2_path))

# Draw matches
fig = plt.figure(figsize=(20, 10))

# Convert confidence to colors (red=low, green=high)
colors = plt.cm.RdYlGn(conf)

# Create side-by-side visualization
h1, w1 = img1.shape[:2]
h2, w2 = img2.shape[:2]
vis = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
vis[:h1, :w1] = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
vis[:h2, w1:] = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

# Draw matches
for i, (pt1, pt2) in enumerate(zip(kpts1, kpts2)):
    color = tuple(int(c * 255) for c in colors[i][:3])
    pt1 = tuple(pt1.astype(int))
    pt2 = tuple((pt2 + [w1, 0]).astype(int))
    cv2.line(vis, pt1, pt2, color, 1)
    cv2.circle(vis, pt1, 3, color, -1)
    cv2.circle(vis, pt2, 3, color, -1)

plt.imshow(vis)
plt.title(f"{len(kpts1)} matches between {img1_path.name} and {img2_path.name}")
plt.axis('off')
plt.show()

## 6. Export Results

Export the reconstruction in various formats.

In [ ]:
# Export results
output_dir = './output/export'
pipeline.export_results(
    reconstruction,
    output_dir,
    formats=['ply', 'json', 'colmap']
)

print(f"Results exported to {output_dir}")
print("Files created:")
for f in Path(output_dir).glob('*'):
    print(f"  - {f.name}")

## 7. Advanced: Compare Different Matchers

Let's compare the performance of different matchers on the same image pair.

In [ ]:
# Compare matchers
matchers = {
    'LoFTR': LoFTRMatcher(device=device),
    'SuperGlue': SuperGlueMatcher(device=device)
}

results = {}
for name, matcher in matchers.items():
    kpts1, kpts2, conf = matcher.match_pair(str(img1_path), str(img2_path))
    results[name] = {
        'num_matches': len(kpts1),
        'mean_conf': np.mean(conf),
        'median_conf': np.median(conf),
        'high_conf': np.sum(conf > 0.5)
    }

# Display comparison
import pandas as pd
df = pd.DataFrame(results).T
print("\nMatcher Comparison:")
print(df)

## 8. Tips for Best Results

1. **Image Quality**: Use high-resolution images with good lighting
2. **Overlap**: Ensure 60-80% overlap between consecutive images
3. **Coverage**: Capture images from multiple viewpoints
4. **Stability**: Avoid motion blur and use stable camera positions
5. **Textures**: Include textured surfaces for better feature matching

## Next Steps

- Try different matcher configurations
- Experiment with test-time augmentation settings
- Use the point cloud for downstream tasks (mesh generation, measurements, etc.)
- Integrate with your own computer vision pipeline